In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [2]:
# Load the dataset from a CSV file
df = pd.read_csv('labeled_twcs.csv')

<ipython-input-2-c7e208d11859>:2: DtypeWarning: Columns (1,2,3,4,5,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('labeled_twcs.csv')


In [3]:
def inbound_val(x):
  if x in ["True","False"]: return x
  elif x in ["True ", "False "]: return x[:-1]
  elif x=="POSITIVE": return "True"
  elif x=="NEGATIVE": return "False"
  else: "NEGATIVE"
df["inbound"] = df["inbound"].apply(lambda x: inbound_val(x))

In [4]:
from sklearn.utils import resample
# Separate positive and negative examples
df_pos = df[df['label'] == 'POSITIVE']
df_neg = df[df['label'] == 'NEGATIVE']

# Downsample negative examples to match positive examples
df_neg_downsampled = resample(df_neg, replace=False, n_samples=len(df_pos), random_state=42)

# Combine the positive and negative examples
df= pd.concat([df_pos, df_neg_downsampled])

In [6]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
# Define the preprocessing steps
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stopwords_set = set(stopwords.words('english'))

def preprocess_text(text):
    text = str(text).lower()
    words = text.split()
    #words = [stemmer.stem(word) for word in words if word not in stopwords_set]
    #words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)
    
    return filtered_text

# Preprocess the text column
df['text'] = df['text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# Encode the text column using TfidfVectorizer
tfidf = TfidfVectorizer()
text_encoded = tfidf.fit_transform(df['text'].fillna(''))

In [8]:
# Encode the author_id, inbound, and label columns using LabelEncoder
le = LabelEncoder()
df['author_id_encoded'] = le.fit_transform(df['author_id'])
df['inbound_encoded'] = le.fit_transform(df['inbound'])
df['label_encoded'] = le.fit_transform(df['label'])

In [9]:
# Split the data into training and testing datasets
from scipy.sparse import csr_matrix, hstack
X = hstack([text_encoded, csr_matrix(df['author_id_encoded']).T, csr_matrix(df['inbound_encoded']).T])
y = df['label_encoded'].apply(lambda x: 1 if x == 1 else 0)  # target variable for positive sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [36]:
# Train Logistic Regression, Multinomial Regression, KNN with target variable
lr = LogisticRegression()
nb = MultinomialNB()

lr.fit(X_train, y_train)
nb.fit(X_train, y_train)

MultinomialNB()

In [37]:
# Evaluate the models based on classification report, f1 score, precision, recall, accuracy, and confusion matrix
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    clf_report = classification_report(y_test, y_pred, output_dict=True)
    cm = confusion_matrix(y_test, y_pred)

    return {
        'Model': model.__class__.__name__,
        'Accuracy': acc,
        'F1 Score (Negative)': clf_report['0']['f1-score'],
        'F1 Score (Positive)': clf_report['1']['f1-score'],
        'Precision (Negative)': clf_report['0']['precision'],
        'Precision (Positive)': clf_report['1']['precision'],
        'Recall (Negative)': clf_report['0']['recall'],
        'Recall (Positive)': clf_report['1']['recall'],
        'Confusion Matrix': cm
    }

In [38]:
evaluations = []

evaluations.append(evaluate_model(lr, X_test, y_test))
evaluations.append(evaluate_model(nb, X_test, y_test))

# Print the evaluation table
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
df_eval = pd.DataFrame(evaluations)
df_eval = df_eval[['Model', 'Accuracy', 'F1 Score (Negative)', 'F1 Score (Positive)', 'Precision (Negative)', 'Precision (Positive)', 'Recall (Negative)', 'Recall (Positive)', 'Confusion Matrix']]
display(df_eval)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Model,Accuracy,F1 Score (Negative),F1 Score (Positive),Precision (Negative),Precision (Positive),Recall (Negative),Recall (Positive),Confusion Matrix
0,LogisticRegression,0.500,0.000,0.667,0.000,0.500,0.000,1.000,"[[0, 30377], [0, 30407]]"
1,MultinomialNB,0.729,0.708,0.747,0.766,0.701,0.659,0.799,"[[20006, 10371], [6116, 24291]]"
